In [166]:
import pandas as pd
import numpy as np
import time
from datetime import datetime
import sys
import csv
import os
from operator import itemgetter, attrgetter
import math

# PRASE DATA

In [223]:
df = pd.read_csv('out.csv', skiprows=1 ,names=['username', 'query_date', 'report_name', 'doc', 'start_date', 'end_date', 'is_period'])
# df = pd.read_csv('out.csv')
weekdaydf = []
daydf = []
monthdf = []
yeardf = []

start_weekdaydf = []
start_monthdf = []
start_yeardf = []
start_daydf = []
end_weekdaydf = []
end_monthdf = []
end_yeardf = []
end_daydf = []

try:
    for i in range(len(df['query_date'])):
        date_obj = datetime.strptime(df['query_date'][i], '%Y-%m-%d')
        weekdaydf.append(int(date_obj.weekday() + 1) )
        daydf.append(int(date_obj.day))
        monthdf.append(int(date_obj.month))
        yeardf.append(int(date_obj.year))
        
    df.loc[:,'query_date_year'] = yeardf
    df.loc[:,'query_date_month'] = monthdf
    df.loc[:,'query_date_day'] = daydf
    df.loc[:,'query_date_weekday'] = weekdaydf
    
    nulldetection = df['start_date'].isnull().values
    for i in range(len(df['start_date'])):
#         print(df['start_date'][i])
    
        if not nulldetection[i]:
            date_obj = datetime.strptime(df['start_date'][i], '%Y-%m-%d')
            start_weekdaydf.append(int(date_obj.weekday() + 1) )
            start_daydf.append(int(date_obj.day))
            start_monthdf.append(int(date_obj.month))
            start_yeardf.append(int(date_obj.year))
        else:
            start_weekdaydf.append(0)
            start_monthdf.append(0)
            start_yeardf.append(0)
            start_daydf.append(0)
          
    df.loc[:,'doc_start_date_year'] = start_yeardf
    df.loc[:,'doc_start_date_month'] = start_monthdf
    df.loc[:,'doc_start_date_day'] = start_daydf
    df.loc[:,'doc_start_date_weekday'] = start_weekdaydf
    
    nulldetection = df['end_date'].isnull().values
    for i in range(len(df['end_date'])):

        if not nulldetection[i] and df['end_date'][i] != '9999-12-31':
            date_obj = datetime.strptime(df['end_date'][i], '%Y-%m-%d')
            end_weekdaydf.append(int(date_obj.weekday() + 1) )
            end_daydf.append(int(date_obj.day))
            end_monthdf.append(int(date_obj.month))
            end_yeardf.append(int(date_obj.year))
        else:
            end_weekdaydf.append(0)
            end_monthdf.append(0)
            end_yeardf.append(0)
            end_daydf.append(0)

    df.loc[:,'doc_end_date_year'] = end_yeardf
    df.loc[:,'doc_end_date_month'] = end_monthdf
    df.loc[:,'doc_end_date_day'] = end_daydf
    df.loc[:,'doc_end_date_weekday'] = end_weekdaydf
except:
    print("Unexpected error:", sys.exc_info())

df.to_csv('prase_out.csv')

In [225]:
df.head()

,username,query_date,report_name,doc,start_date,end_date,is_period,query_date_year,query_date_month,query_date_day,query_date_weekday,doc_start_date_year,doc_start_date_month,doc_start_date_day,doc_start_date_weekday,doc_end_date_year,doc_end_date_month,doc_end_date_day,doc_end_date_weekday
0,001713,2017-02-10,001713,ODS_UNB_0460,2016-08-31,2016-08-31,True,2017,2,10,5,2016,8,31,3,2016,8,31,3
1,calvin,2017-02-10,calvin,ODS_PA_0500_P,2016-11-30,2016-11-30,True,2017,2,10,5,2016,11,30,3,2016,11,30,3
2,calvin,2017-02-10,calvin,ODS_FIN_0573,NaN,9999-12-31,True,2017,2,10,5,0,0,0,0,0,0,0,0
3,user1,2017-02-10,user1,ODS_POS_0990,NaN,9999-12-31,True,2017,2,10,5,0,0,0,0,0,0,0,0
4,user1,2017-02-10,user1,ODS_POS_0990,NaN,9999-12-31,True,2017,2,10,5,0,0,0,0,0,0,0,0


In [226]:
df.shape

(32323, 19)

In [227]:
# for total data
cwd = os.getcwd()
TotalFolderpath = '/Total_statistic/'
folderName = cwd + TotalFolderpath

if not os.path.exists(folderName):
    os.makedirs(folderName)

user = df['username'].value_counts(0)
doc = df['doc'].value_counts(0)
user_query_date = df['query_date'].value_counts(0)

user.to_csv(folderName + 'user_statistic.csv')
doc.to_csv(folderName + 'doc_statistic.csv')
user_query_date.to_csv(folderName + 'query_date_statistic.csv')

In [228]:
# for different type data
each_user = user.axes[0]
each_doc = doc.axes[0]

print("Active users :", len(each_user))
print("Active docs  :", len(each_doc))

Active users : 101
Active docs  : 613


In [229]:
def check_file_path_exist(csvName):
    if not os.path.exists(os.path.dirname(csvName)):
        try:
            os.makedirs(os.path.dirname(csvName))
        except OSError as exc: # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise

In [230]:
# for user type
cwd = os.getcwd()
UserFolderpath = '/User_statistic/'
UserClinkFolderpath = '/UserClick_statistic/'

csvFinalName = '_statistic.csv'
UserQueryCSV = 'UserQueryData_statistic.csv'
UserClinkCSV = 'UserClickDoc_statistic.csv'
UserEachDayQueryCSV = 'QueryDoc_statistic.csv'
UserEachDayQueryAllCSV = 'QueryDocAll_statistic.csv'

user_Doc_StaticResult = [];

for idx, user_i in enumerate(each_user):
    if(idx%10==0):
        print("Processing: {0:.2f}%".format(idx*100/len(each_user)))
    try:
        csvName = cwd + UserFolderpath + str(user_i) + csvFinalName
        check_file_path_exist(csvName)
        userDF = df.loc[df['username'] == user_i]
        userDF.to_csv(csvName)

        # prase for each user : click doc counter for all time
        csvClickDocName = cwd + UserFolderpath + str(user_i)  + '/' + UserClinkCSV
        check_file_path_exist(csvClickDocName)
        userDF_Doc_counter = userDF['doc'].value_counts(0)
        userDF_Doc_counter.to_csv(csvClickDocName)

        # prase for each user : click doc counter for each day
        csvQueryDate = cwd + UserFolderpath + str(user_i) + '/' + UserQueryCSV
        check_file_path_exist(csvQueryDate)
        userDF_each_date = userDF['query_date'].value_counts(0)
        userDF_each_date.to_csv(csvQueryDate)

        # prase for each user : click each doc counter for each day

    #     user_each_data = userDF_each_date.axes[0]
    #     for user_each_data_i in user_each_data:
    #         # save all query data in each date
    #         userDF_each_data_doc = userDF.loc[userDF['query_date'] == user_each_data_i]
    #         csvEachDayQueryAllDoc = cwd + UserFolderpath + str(user_i) + '/' + str(user_each_data_i) + '/' + UserEachDayQueryAllCSV
    #         check_file_path_exist(csvEachDayQueryAllDoc)
    #         userDF_each_data_doc.to_csv(csvEachDayQueryAllDoc)
    #         print(userDF_each_data_doc)

    #         # save query doc data couter in each date
    #         user_each_data_doc_query = userDF_each_data_doc['doc'].value_counts(0)
    #         csvEachDayQueryDoc = cwd + UserFolderpath + str(user_i) + '/' + str(user_each_data_i) + '/' + UserEachDayQueryCSV
    #         check_file_path_exist(csvEachDayQueryDoc)
    #         user_each_data_doc_query.to_csv(csvEachDayQueryDoc)

       
        
        user_each_doc = userDF_Doc_counter.axes[0]
        for user_each_doc_i in user_each_doc:
            # save all query data in each date
            userDF_each_data_doc = userDF.loc[userDF['doc'] == user_each_doc_i]
            csvEachDayQueryAllDoc = cwd + UserFolderpath + str(user_i) + '/' + str(user_each_doc_i) + '/' + UserEachDayQueryAllCSV
            check_file_path_exist(csvEachDayQueryAllDoc)
            userDF_each_data_doc.to_csv(csvEachDayQueryAllDoc)
#             print(userDF_each_data_doc)

            # save query doc data couter in each date
            user_each_data_doc_query = userDF_each_data_doc['query_date'].value_counts(0)
            csvEachDayQueryDoc = cwd + UserFolderpath + str(user_i) + '/' + str(user_each_doc_i) + '/' + UserEachDayQueryCSV
            check_file_path_exist(csvEachDayQueryDoc)
            user_each_data_doc_query.to_csv(csvEachDayQueryDoc)
            
        user_Doc_StaticResult.append([str(user_i),len(user_each_doc)])
        
    except:
        print("Unexpected error:", sys.exc_info())

        

Processing: 0.00%
Processing: 9.90%
Processing: 19.80%
Processing: 29.70%
Processing: 39.60%
Processing: 49.50%
Processing: 59.41%
Processing: 69.31%
Processing: 79.21%
Processing: 89.11%
Processing: 99.01%


In [231]:
user_Doc_SortStaticResult = sorted(user_Doc_StaticResult, key=itemgetter(1),reverse=True)
print("statistic result : ", user_Doc_SortStaticResult[:])

# click doc type rate < 20 , show all doc
# click doc type rate > 20 , predic possible option

statistic result :  [['mPOS', 195], ['user1', 135], ['000358', 126], ['yusiang', 124], ['user2', 97], ['user4', 94], ['user18', 90], ['user10', 84], ['user19', 83], ['user3', 82], ['ricky', 78], ['roxana', 77], ['calvin', 75], ['shiang', 73], ['user5', 71], ['002064', 66], ['user15', 57], ['000368', 51], ['000898', 47], ['user8', 44], ['user14', 40], ['CSIS', 36], ['000442', 35], ['000232', 34], ['CSIS-SIT3', 33], ['jesse', 31], ['000868', 30], ['001415', 28], ['001231', 28], ['000718', 26], ['000923', 24], ['weiting', 23], ['000768', 22], ['001867', 20], ['CSIS-R5', 20], ['user7', 19], ['jimmy', 17], ['001713', 16], ['000791', 16], ['000279', 14], ['dennis', 14], ['jessie', 14], ['001751', 14], ['user9', 13], ['000316', 12], ['002266', 12], ['esp.admin', 12], ['001980', 12], ['000761', 11], ['user6', 10], ['000802', 10], ['000124', 9], ['000443', 9], ['001698', 9], ['000217', 9], ['user11', 8], ['000219', 8], ['user13', 8], ['000759', 8], ['001340', 8], ['001993', 8], ['user12', 7], [

In [232]:
# for doc type
DocFolderpath = '/Doc_statistic/'
DocClickFolderpath = '/DocClick_statistic/'

for doc_i in each_doc:
    
    csvName = cwd + DocFolderpath + str(doc_i) + csvFinalName
    csvClickName = cwd + DocClickFolderpath + str(doc_i) + csvFinalName
    check_file_path_exist(csvName)
    check_file_path_exist(csvClickName)
    
    docDF = df.loc[df['doc'] == doc_i]
#     print(doc_i)
    docDF_counter = docDF['username'].value_counts(0)
#     print(doc_i)
    docDF.to_csv(csvName)
    docDF_counter.to_csv(csvClickName)